Вам представлены данные о доходах сотрудников участка горячего проката за год, в том числе данные, описывающие специфику рабочих мест.
Доход состоит из оклада, премии, доплат и индивидуальных надбавок. Данных об их размерах не приводится, так как именно их Вам предстоит восстановить.
Задание 
Выявите формулу дохода в целом и его составных частей
Проверьте, что данная формула объясняет доход с точностью до рубля
Выявите зависимости между составными частями дохода

In [80]:
#ИМПОРТИРУЕМ БИБЛИОТЕКИ. СЧИТАЕМ ДАТАФРЕЙМ. ПЕРЕИМЕНУЕМ КОЛОНКИ. 
import pandas as pd 
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, max_error
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
import nbconvert
import copy

df = pd.read_excel('df.xlsx')
df=df.rename(columns = {'Вредность (балл.)':'INS','Объем производства':'VOL','Доход':'INCOME','Месяц':'MONTH','Профессия':'PROF','Разряд':'RANK','Оборудование':'TOOL'})

In [81]:
#СГРУППИРУЕМ ПО ОБЪЕМУ И ПОСМОТРИМ НА СРЕДНИЙ ДОХОД ДЛЯ КАЖДОГО ИЗ НИХ
print(df.groupby('VOL', as_index=0).agg({'INCOME':'mean'}).sort_values(by='VOL',ascending=0))
#ДОХОД НАЧИНАЕТ РАСТИ ЛИШЬ С ОПРЕДЕЛЕННОГО ПОРОГОВОГО ЗНАЧЕНИЯ (ПЛАНА).


     VOL        INCOME
11  1400  25400.345956
10  1250  23896.409559
9   1200  23395.097426
8   1180  23194.572574
7   1080  22191.948309
6   1020  21590.373750
5    950  20888.536765
4    820  19885.912500
3    810  19885.912500
2    790  19885.912500
1    780  19885.912500
0    580  19885.912500


In [82]:
#ВЫЧИСЛИМ ИНДИВИДУАЛЬНЫЕ НАДБАВКИ. ДЛЯ ЭТОГО СОЗДАДИМ ПЕРЕМЕННУЮ ALL, КОТОРАЯ БУДЕТ ПОСТРОЧНОЙ СУММОЙ ВСЕХ ПАРАМЕТРОВ В i-ОЙ СТРОКЕ.
#ТАКИМ ОБРАЗОМ, ПЕРЕМЕННАЯ ALL ПРИМЕТ ЗНАЧЕНИЯ ВСЕХ ВОЗМОЖНЫХ КОМБИНАЦИНАЦИЙ ПАРАМЕТРОВ. ТЕПЕРЬ ДОХОД МОЖНО ПРЕДСКАЗАТЬ С ТОЧНОСТЬЮ ДО РУБЛЯ 2МЯ ФИКТИВНЫМИ ПЕРЕМЕННЫМИ 
#ALL И FIO. СОЗДАДИМ ТАКИЕ ПЕРЕМЕННЫЕ И ИСПОЛЬЗУЕМ ИХ В КАЧЕСТВЕ ПРЕДИКТОРОВ ЛИНЕЙНОЙ РЕГРЕССИИ С ТРЕНИРОВОЧНОЙ ПЕРЕМЕННОЙ 'ДОХОД' ДЛЯ СТРОК, ПРИНАДЛЕЖАЩИХ
#МЕСЯЦУ, ДЛЯ КОТОРОГО ПЛАН НЕ ПЕРЕВЫПОЛНЕН. КОЭФФИЦИЕНТЫ ПРИ ПЕРЕММЕННЫХ FIO ЗАПИШЕМ В ИСХОДНУЮ ТАБЛИЦУ, В КОЛОНКУ IND. ЭТО И ЕСТЬ ИНДИВИДУАЛЬНЫЕ НАДБАВКИ. 

df['ALL'] = df.PROF + ' ' + df.TOOL + ' ' + df.RANK.astype(str) + ' ' + df.INS.astype(str)
df1=copy.copy(df[df.VOL==580])
Y = df1.INCOME
X = pd.get_dummies(df1.ALL, drop_first=0,prefix='ALL')
X=X.join(pd.get_dummies(df1.FIO, drop_first=0))

model = LinearRegression(fit_intercept=0,positive=1)
model.fit(X, Y)

Y_pred = model.predict(X)

print("Root Mean squared error: %.2f" % mean_squared_error(Y_pred,Y,squared=0))
print("Max_error: %.2f" % max_error(Y_pred,Y))
print("Coefficient of determination: %.2f" % r2_score(Y, Y_pred))
coef=[model.coef_[i] for i in range(model.coef_.size)]

IND=pd.DataFrame(columns={'IND','FIO'})
IND.FIO=df.FIO.unique()
IND.IND=coef[22:]

df['IND']=0
 
for i in range (2400):
    for n in range (200):
        if df.loc[i,'FIO']==IND.loc[n,'FIO']:
            df.loc[i,'IND']=IND.loc[n,'IND']
        else:
            pass

del(df['ALL'])
display(df)

Root Mean squared error: 0.00
Max_error: 0.00
Coefficient of determination: 1.00


,FIO,MONTH,PROF,RANK,TOOL,INS,VOL,INCOME,IND
0,Абрамов Е. Д.,2009-01-01,Уборщик,3,Пилигримовые станы,18,580,16638.000000,1218.0
1,Абрамов Е. Д.,2009-02-01,Уборщик,3,Пилигримовые станы,18,790,16638.000000,1218.0
2,Абрамов Е. Д.,2009-03-01,Уборщик,3,Пилигримовые станы,18,820,16638.000000,1218.0
3,Абрамов Е. Д.,2009-04-01,Уборщик,3,Пилигримовые станы,18,1200,19396.411765,1218.0
4,Абрамов Е. Д.,2009-05-01,Уборщик,3,Пилигримовые станы,18,1250,19790.470588,1218.0
...,...,...,...,...,...,...,...,...,...
2395,Яковлев Д. И.,2009-08-01,Подручный вальцовщика,4,Прошивной стан,18,1180,21800.170588,1413.0
2396,Яковлев Д. И.,2009-09-01,Подручный вальцовщика,4,Прошивной стан,18,1080,20885.876471,1413.0
2397,Яковлев Д. И.,2009-10-01,Подручный вальцовщика,4,Прошивной стан,18,780,18783.000000,1413.0
2398,Яковлев Д. И.,2009-11-01,Подручный вальцовщика,4,Прошивной стан,18,1020,20337.300000,1413.0


In [83]:
#ВЫЯСНИЛОСЬ, ЧТО ДЛЯ ЛЮБОГО ЧЕЛОВЕКА ДОХОД ЛИНЕЙНО ЗАВИСИТ ОТ ОБЪЕМА, ЕСЛИ ОБЪЕМ БОЛЬШЕ 820. РАСЧИТАЕМ КОЭФФИЦИЕНТЫ ЛИНЕЙНОЙ РЕГРЕССИИ ДЛЯ ОБЪЕМА В РОЛИ ПРЕДИКТОРА
#И ДОХОДА В РОЛИ ТРЕНЕРОВОЧНОЙ ПЕРЕМЕННОЙ ДЛЯ КАЖДОГО ИМЕНИ. СОХРАНИМ ПОЛУЧИВШИЙСЯ КОЭФФИЦИЕНТ ПРИ ОБЪЕМЕ И СВОБОДНЫЙ ЧЛЕН. ЗАТЕМ БУДЕТ ИСПОЛЬЗОВАТЬ ИХ В РОЛИ 
#ТРЕНЕРОВОЧНЫХ ПЕРЕМЕННЫХ, ЧТОБЫ ВЫЯСНИТЬ НА КАКУЮ ЧАСТЬ ДОХОДА НАЗНАЧАЕТСЯ ПРЕМИЯ ЗА ОБЪЕМ И НАЧИНАЯ С КАКОГО ПОРОГОВОГО ЗНАЧЕНИЯ.ОТ ЛЮБОЙ РЕГРЕССИОННОЙ МОДЕЛИ 
#В ДАННОЙ ЗАДАЧЕ МЫ ХОТИМ ДОБИТСЯ ЕДИНИЧНОГО КОЭФФИЦИЕНТА ДЕТЕРМИНАЦИИ И 0ОЙ СРЕДНЕКВАДРАТИЧНОЙ ОШИБКИ.

df['coef']=0
df['intercept']=0
for i in (df.FIO.unique()):
    df2=copy.copy(df[df.FIO==i].query('VOL>900'))
    Y = df2.INCOME
    X = df2[['VOL']]
    model = LinearRegression(fit_intercept=1,positive=1)
    model.fit(X, Y)
    intercept=model.intercept_
    coef=model.coef_
    for n in list(df[df.FIO==i].query('VOL>900').index):
        df.loc[n,'coef']=coef
        df.loc[n,'intercept']=intercept
    coef=0
    intercept=0

In [84]:
df2=copy.copy(df[df.VOL>900]).reset_index(drop=0).query('RANK<5')

Y = df2.coef
X = df2[['RANK','IND']]
X=X.join(pd.get_dummies(df2.PROF, drop_first=0))
model = LinearRegression(fit_intercept=0,positive=1)
model.fit(X, Y)
Y_pred = model.predict(X)
coef=[model.coef_[i] for i in range(model.coef_.size)]

print("Root Mean squared error: %.2f" % mean_squared_error(Y_pred,Y,squared=0))
print("Max_error: %.2f" % max_error(Y_pred,Y))
print("Coefficient of determination: %.2f" % r2_score(Y, Y_pred))
print(coef,end=' ')


Root Mean squared error: 0.00
Max_error: 0.00
Coefficient of determination: 1.00
[1.1470588235293704, 0.0005882352941176093, 3.7235294117649325, 3.7235294117649547, 3.723529411764968, 3.7235294117648703, 3.7235294117649165, 3.723529411764938, 3.723529411764804] 

In [85]:
#ЗАМЕТИМ ЧТО КОЭФФИЦИЕНТЫ ПРИ РАНГЕ И ИНДИВИДУАЛЬНЫХ НАДБАВКАХ МЕНЬШЕ РОВНО В 2 РАЗА ПРИ ТАРГЕТИРОВАНИИ СВОБОДНОГО ЧЛЕНА ПОЛУЧЕННОГО ИЗ РЕГРЕССИИ С ОБЪЕМОМ,
#НЕЖЕЛИ ПРИ ТАРГЕТИРОВАНИИ ДОХОДА ТЕМИ ЖЕ ПРЕДИКТОРАМИ ДЛЯ МЕСЯЦЕВ С НЕ ПЕРЕВЫПОЛНЕННЫМ ПЛАНОМ. КОЭФФИЦИЕНТ ПРИ ВРЕДНОСТИ ПРИ ЭТОМ ОДИНАКОВ. ЗНАЧИТ,
#ПРЕМИЯ ЗА ОБЪЕМ НАЧИСЛЯЕТСЯ НА ПОЛОВИНУ ОКЛАДА И ИНД НАДБАВОК.

df3=copy.copy(df[df.VOL>900]).query('RANK<5')

Y = df3.intercept
X = df3[['RANK','IND','INS']]
X=X.join(pd.get_dummies(df3.PROF, drop_first=0))


model = LinearRegression(fit_intercept=0,positive=1)
model.fit(X, Y)
Y_pred = model.predict(X)
coef=[model.coef_[i] for i in range(model.coef_.size)]

print("Root Mean squared error: %.2f" % mean_squared_error(Y_pred,Y,squared=0))
print("Max_error: %.2f" % max_error(Y_pred,Y))
print("Coefficient of determination: %.2f" % r2_score(Y, Y_pred))
print(coef,end=' ')
print(intercept,end=' ')

Root Mean squared error: 0.00
Max_error: 0.00
Coefficient of determination: 1.00
[975.0000000000617, 0.5000000000000646, 179.99999999999864, 3164.9999999997162, 3164.9999999996876, 3164.9999999996835, 3164.999999999742, 3164.9999999996935, 3164.999999999688, 3164.999999999804] 0 

In [86]:
#КАРТИНА МЕНЯЕТСЯ ПРИ РАНГЕ БОЛЬШЕ ИЛИ РАВНОМ 5. 

df3=copy.copy(df[df.VOL<900]).query('RANK>=5')

Y = df3.INCOME
X = df3[['RANK','IND','INS']]
X=X.join(pd.get_dummies(df3.TOOL, drop_first=0))

model = LinearRegression(fit_intercept=0,positive=1)
model.fit(X, Y)
Y_pred = model.predict(X)
coefT=[model.coef_[i] for i in range(model.coef_.size)]

print("Root Mean squared error: %.2f" % mean_squared_error(Y_pred,Y,squared=0))
print("Max_error: %.2f" % max_error(Y_pred,Y))
print("Coefficient of determination: %.2f" % r2_score(Y, Y_pred))
print(coefT,end=' ')
print(intercept,end=' ')


Root Mean squared error: 0.00
Max_error: 0.00
Coefficient of determination: 1.00
[3039.999999999966, 0.9999999999999857, 180.00000000000173, 1190.0000000001419, 1190.0000000001467, 780.0000000002033, 780.0000000002167, 1190.0000000001362, 1190.0000000001467, 780.0000000002192, 780.0000000002126] 0 

In [87]:
TPREM=pd.DataFrame(columns={'PREMIUM','TOOL'})
TPREM.TOOL=sorted(df3.TOOL.unique())
TPREM.PREMIUM=coefT[3:]

df['TPREM']=0
 
for i in range (2400):
    for n in range (TPREM.TOOL.size):
        if df.loc[i,'TOOL']==TPREM.loc[n,'TOOL'] and df.loc[i,'RANK']>=5:
            df.loc[i,'TPREM']=TPREM.loc[n,'PREMIUM']
        else:
            pass
#display(df[df.RANK>=5])
TPREM

,PREMIUM,TOOL
0,1190.0,Калибровочный стан
1,1190.0,"Калибровочный стан, ГРП"
2,780.0,Контрольно-измерительная аппаратура
3,780.0,Нагревательные печи
4,1190.0,"Нагревательные печи, кантовка"
5,1190.0,"Нагревательные печи, нагрев"
6,780.0,Пилигримовые станы
7,780.0,Прошивной стан


In [88]:
#ДЛЯ УДОБСТВА ПРОВЕРКИ ЗАМЕНИМ ВСЕ ОБЪЕМЫ МЕНЬШЕ 850 на 850
def to_logical_2 (x):
    if x<=850:  
        return 850
    else:
        return x
    
df.VOL=df.VOL.apply(to_logical_2)

In [89]:
#ПРОВЕРКА
for i in range (2400):
    if df.loc[i,'RANK']<5:
        INCOME_pred = (1950*df.loc[i,'RANK']+df.loc[i,'IND']+6330)*(0.5+0.5*df.loc[i,'VOL']/850)+180*df.loc[i,'INS']
        if abs(INCOME_pred-df.loc[i,'INCOME'])>1:
            print ('god damn it')
    if df.loc[i,'RANK']>=5:
        INCOME_pred = (3040*df.loc[i,'RANK']+df.loc[i,'IND']+df.loc[i,'TPREM'])*(0.5+0.5*df.loc[i,'VOL']/850)+180*df.loc[i,'INS']
        if abs(INCOME_pred-df.loc[i,'INCOME'])>1:
            print ('god damn it')

In [90]:
del(df['coef'])
del(df['intercept'])

In [91]:
df.to_excel("output.xlsx")